In [5]:
import random
import numpy as np

In [11]:
def get_final_board_state(moves, board_size):
    board = np.zeros((board_size, board_size), dtype=int)
    for i, move in enumerate(moves):
        player = 1 if i % 2 == 0 else 2
        row, col = move // board_size, move % board_size
        board[row, col] = player
    # print("THE BOOOOOARD: ", board)
    return board.tolist()


In [36]:
BOARD_DIM = 11
# Define neighbors for hexagonal connections
neighbors = [-(BOARD_DIM+2) + 1, -(BOARD_DIM+2), -1, 1, (BOARD_DIM+2), (BOARD_DIM+2) - 1]

# Define the structure of the Hex game as a class
class HexGame:
    def __init__(self):
        # The board is initialized as a 2D array (flattened for convenience)
        self.board = np.zeros(((BOARD_DIM+2)*(BOARD_DIM+2)*2), dtype=int)
        self.open_positions = [(i*(BOARD_DIM + 2) + j) for i in range(1, BOARD_DIM+1) for j in range(1, BOARD_DIM+1)]
        self.number_of_open_positions = BOARD_DIM * BOARD_DIM
        self.moves = np.zeros(BOARD_DIM * BOARD_DIM, dtype=int)
        self.connected = np.zeros(((BOARD_DIM+2)*(BOARD_DIM+2)*2), dtype=int)
        self._init_board()

    # Initialize the game board
    def _init_board(self):
        for i in range(BOARD_DIM+2):
            for j in range(BOARD_DIM+2):
                if i == 0:
                    self.connected[(i*(BOARD_DIM + 2) + j) * 2] = 1
                if j == 0:
                    self.connected[(i*(BOARD_DIM + 2) + j) * 2 + 1] = 1

    # Check if the board is full
    def full_board(self):
        return self.number_of_open_positions == 0

    # Place a piece randomly
    def place_piece_randomly(self, player):
        random_empty_position_index = random.randint(0, self.number_of_open_positions - 1)
        empty_position = self.open_positions[random_empty_position_index]

        self.board[empty_position * 2 + player] = 1

        # Convert the actual board position to the relative index in a BOARD_DIM x BOARD_DIM grid
        relative_position = (empty_position // (BOARD_DIM + 2) - 1) * BOARD_DIM + (empty_position % (BOARD_DIM + 2) - 1)
        self.moves[BOARD_DIM * BOARD_DIM - self.number_of_open_positions] = relative_position

        # Update the list of open positions
        self.open_positions[random_empty_position_index] = self.open_positions[self.number_of_open_positions - 1]
        self.number_of_open_positions -= 1

        return empty_position

    # Check if a player has won
    def winner(self, player, position):
        for i in range(6):
            neighbor = position + neighbors[i]
            if self.connected[neighbor * 2 + player]:
                return True
        return False

    # Print the game board
    def print_board(self):
        for i in range(BOARD_DIM):
            print(" " * i, end="")
            for j in range(BOARD_DIM):
                if self.board[((i+1)*(BOARD_DIM+2) + j + 1)*2] == 1:
                    print(" X", end="")
                elif self.board[((i+1)*(BOARD_DIM+2) + j + 1)*2 + 1] == 1:
                    print(" O", end="")
                else:
                    print(" ·", end="")
            print()


In [37]:
def hex_winner(matrix):
    n = len(matrix)
    
    # Directions for hexagonal neighbors: 6 possible directions
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, 1), (1, -1)]
    
    def is_valid(x, y, player):
        # Check if the cell (x, y) is within bounds, belongs to the player, and is unvisited
        return 0 <= x < n and 0 <= y < n and matrix[x][y] == player and not visited[x][y]
    
    def dfs(x, y, player):
        # Debug statement
        # print(f"Visiting cell ({x}, {y}) for Player {player}")
        
        if player == 1 and x == n - 1:  # Player 1 wins (reached bottom row)
            return True
        if player == 2 and y == n - 1:  # Player 2 wins (reached right column)
            return True
        
        visited[x][y] = True
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid(nx, ny, player) and dfs(nx, ny, player):
                return True
        
        return False
    
    # Check for Player 1 (top to bottom)
    visited = [[False] * n for _ in range(n)]
    for y in range(n):
        if matrix[0][y] == 1:
            # print(f"Starting DFS for Player 1 at (0, {y})")
            if dfs(0, y, 1):
                return True, 1 #"Player 1 wins"
    
    # Check for Player 2 (left to right)
    visited = [[False] * n for _ in range(n)]
    for x in range(n):
        if matrix[x][0] == 2:
            # print(f"Starting DFS for Player 2 at ({x}, 0)")
            if dfs(x, 0, 2):
                return True, 2 #"Player 2 wins"
    
    return False, 0 # no winner

In [38]:
# # leave for now 
# with open("hex_game_results_13.csv", "w") as output_file:
#         output_file.write("Game ID,Winner,Moves\n")
#         for game in range(100):
#             hg = HexGame()
#             player = 0
#             winner = -1

#             # Simulate the game
#             while not hg.full_board():
#                 position = hg.place_piece_randomly(player)
#                 if hg.winner(player, position):
#                     winner = player
#                     break
#                 player = 1 - player  # Switch player

#             # Write the result to the file
#             output_file.write(f"{game},{winner},")
#             output_file.write(" ".join(map(str, hg.moves[:BOARD_DIM*BOARD_DIM - hg.number_of_open_positions])))
#             output_file.write("\n")

#             if game % 100000 == 0:
#                 print(f"Game {game} completed")


In [39]:
with open("hex_game_results_13.csv", "w") as output_file:
        output_file.write("Game ID,Winner,Moves\n")
        for game in range(10):
            hg = HexGame()
            player = 0
            winner = -1

            # Simulate the game
            while not hg.full_board():
                position = hg.place_piece_randomly(player)
                resulting_matrix=get_final_board_state(hg.moves[:BOARD_DIM*BOARD_DIM - hg.number_of_open_positions], BOARD_DIM)
                winner_detected, winner= hex_winner(resulting_matrix)
                if winner_detected:
                     print("The winner is ", winner)
                     print("retulting matrix is", resulting_matrix)

                     break 
                
                     
                # if hg.winner(player, position):
                #     winner = player
                #     break
                player = 1 - player  # Switch player

            # Write the result to the file
            output_file.write(f"{game},{winner},")
            output_file.write(" ".join(map(str, hg.moves[:BOARD_DIM*BOARD_DIM - hg.number_of_open_positions])))
            output_file.write("\n")

            if game % 100000 == 0:
                print(f"Game {game} completed")


The winner is  2
retulting matrix is [[0, 0, 0, 1, 1, 1, 2, 1], [0, 2, 0, 2, 0, 1, 0, 1], [0, 2, 2, 2, 2, 2, 1, 0], [0, 2, 0, 2, 1, 2, 2, 2], [1, 2, 0, 2, 1, 1, 2, 0], [2, 2, 1, 1, 1, 1, 1, 2], [1, 2, 1, 2, 1, 1, 2, 2], [1, 0, 2, 1, 1, 1, 2, 1]]
Game 0 completed
The winner is  1
retulting matrix is [[2, 0, 2, 2, 2, 1, 1, 1], [0, 1, 1, 1, 1, 1, 2, 2], [2, 1, 2, 1, 1, 0, 2, 0], [1, 1, 2, 2, 2, 2, 0, 2], [2, 1, 1, 0, 2, 0, 1, 1], [2, 2, 1, 0, 0, 1, 0, 2], [0, 1, 1, 0, 2, 1, 2, 1], [1, 2, 1, 2, 2, 2, 0, 1]]
The winner is  2
retulting matrix is [[1, 2, 1, 0, 2, 2, 2, 2], [1, 2, 0, 0, 1, 1, 1, 2], [1, 1, 0, 1, 0, 1, 2, 0], [0, 1, 1, 1, 0, 2, 2, 0], [1, 2, 2, 2, 2, 0, 2, 1], [2, 1, 1, 2, 2, 1, 1, 1], [2, 2, 1, 1, 2, 1, 2, 1], [0, 0, 2, 2, 1, 2, 1, 2]]
The winner is  2
retulting matrix is [[0, 2, 1, 1, 2, 1, 0, 1], [2, 1, 1, 2, 0, 1, 1, 2], [1, 0, 1, 1, 0, 2, 2, 2], [1, 2, 1, 2, 1, 1, 2, 2], [2, 0, 1, 0, 0, 2, 1, 0], [1, 1, 2, 1, 2, 2, 1, 2], [2, 2, 2, 2, 1, 1, 0, 2], [0, 2, 1, 0, 2, 1, 1, 2]]